# Live Data Reduction


## From Terminal

You can run the ``beamlime`` command from your terminal to run the application and see the real-time update of a plot.

```bash
beamlime --image-path beamlime_plot.png
```

This command will save the plot into ``beamlime_plot.png``.

## Jupyter Environment

You can also run the same thing in jupyter environment
and use ``plopp`` matplotlib widget to see the real-time updated plot.

**It uses the same ``PlotHandler`` as above,**
**so running this application will create a file called ``beamlime_plot.png`` in the current directory.**

In [ ]:
%matplotlib widget
import scipp as sc

sc.get_logger().addHandler(sc.logging.make_widget_handler())
sc.display_logs()

In [ ]:
from beamlime.constructors import multiple_constant_providers, SingletonProvider
from beamlime.applications.handlers import (
    DataReductionHandler,
    WorkflowResultUpdate,
    PlotStreamer,
)
from beamlime.applications.daemons import DetectorDataReceived, FakeListener, NexusTemplatePath
from beamlime.applications.base import Application
from beamlime.executables.prototypes import default_prototype_factory
from beamlime.logging import BeamlimeLogger
from beamlime.applications._parameters import (
    EventRate,
    NumPixels,
    NumFrames,
    DataFeedingSpeed,
)
from beamlime.stateless_workflow import Workflow
from matplotlib import pyplot as plt

import logging

prototype_factory = default_prototype_factory()
# Replace the default data reduction handler provider with a singleton provider.
# So that we can access to the figure that it is using to plot the histogram.
prototype_factory.providers.pop(PlotStreamer)  # Remove the default handler.
prototype_factory.providers[PlotStreamer] = SingletonProvider(PlotStreamer)

with multiple_constant_providers(
    prototype_factory,
    {
        BeamlimeLogger: sc.get_logger(),
        DataFeedingSpeed: DataFeedingSpeed(1/14),
        EventRate: EventRate(10_000),
        NumPixels: NumPixels(10_000),
        NumFrames: NumFrames(5),
        Workflow: Workflow('dummy'),
        NexusTemplatePath: NexusTemplatePath('../../tests/applications/ymir.json')
    },
):
    prototype_factory[BeamlimeLogger].setLevel(logging.INFO)

    # Build the application
    app = prototype_factory[Application]
    # Register Handlers
    plot_saver = prototype_factory[PlotStreamer]
    app.register_handling_method(WorkflowResultUpdate, plot_saver.update_histogram)
    data_reduction_handler = prototype_factory[DataReductionHandler]
    app.register_handling_method(DetectorDataReceived, data_reduction_handler.process_message)
    # Register Daemons
    app.register_daemon(prototype_factory[FakeListener])

    app.run()
    plt.close()  # Close the plot from this cell.

In [ ]:
from plopp.widgets import Box, HBar

Box(HBar(list(plot_saver.figures.values())))